In [6]:
# %%capture
# # Colab setup
# !git clone https://github.com/leemnj/iterative-dna-reconstruction.git
# %cd iterative-dna-reconstruction
%pip install -e .
%pip -q install biopython

from pathlib import Path
DATA_DIR = Path("../data")
DATA_DIR.mkdir(exist_ok=True)


Obtaining file:///Users/leeminjae/Library/Mobile%20Documents/com~apple~CloudDocs/Desktop/01_Projects/iterative-dna-reconstruction/01_generate-seq-and-emb
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for idr (pyproject.toml) ... done
  Created wheel for idr: filename=idr-0.1.0-0.editable-py3-none-any.whl size=1251 sha256=e460b4e2dff5190d9b7fe0599580dc8b318ad60a2adcee346ff335d8c713186c
  Stored in directory: /private/var/folders/wm/m60tntks4_55jf79w27yb1t40000gn/T/pip-ephem-wheel-cache-_geh1gr6/wheels/77/31/f5/e022ae50055931bbdccac43ccb69836d51684cb9883293e081
Successfully built idr
  Attempting uninstall: idr
    Found existing installation: idr 0.1.0
    Uninstalling idr-0.1.0:
      Successfully uninstalled idr-0.1.0
Note: you may need to restart the kernel to use updated packages.
Note: you may need to r

In [7]:
import gc
from pathlib import Path
import pandas as pd
import torch
import requests

from idr.utils import get_device, iter_models
from idr.preparation import DEFAULT_DECODING_STRATEGIES

print('✅ imports loaded')


✅ imports loaded


In [8]:
ITERATIONS = 30
MASK_RATIO = 0.2
PROMOTER_TSS_LENGTH = 1000
ENSEMBL_SPECIES = 'human'
ENSEMBL_ASSEMBLY = 'GRCh38'

# RESULTS_DIR = Path('data')
PROMOTER_DIR = DATA_DIR / 'promoters'
PROMOTER_DIR.mkdir(parents=True, exist_ok=True)

model_configs = {
    'DNABERT-2': 'zhihan1996/DNABERT-2-117M',
    'NT-v2-50m': 'InstaDeepAI/nucleotide-transformer-v2-50m-multi-species',
    'NT-v2-500m': 'InstaDeepAI/nucleotide-transformer-v2-500m-multi-species',
}

torch_dtype = 'float32'
model_labels = list(model_configs.keys())

print(f'ITERATIONS={ITERATIONS}, MASK_RATIO={MASK_RATIO}')
print(f'PROMOTER_TSS_LENGTH={PROMOTER_TSS_LENGTH}')
print(f'모델: {model_labels}')


ITERATIONS=30, MASK_RATIO=0.2
PROMOTER_TSS_LENGTH=1000
모델: ['DNABERT-2', 'NT-v2-50m', 'NT-v2-500m']


In [9]:
GENE_NAME = 'HBB'
# GTF_FEATHER_PATH = Path('../dna-fm-to-alphagenome/gencode.v46.annotation.gtf.gz.feather')
GTF_FEATHER_PATH = DATA_DIR / 'gencode.v46.annotation.gtf.gz.feather'

print(f'Gene target: {GENE_NAME}')
print(f'GTF path: {GTF_FEATHER_PATH}')


Gene target: HBB
GTF path: ../data/gencode.v46.annotation.gtf.gz.feather


## Finding promoter site
GTF feather 파일에서 가져오기


In [10]:
if not GTF_FEATHER_PATH.exists():
    raise FileNotFoundError(
        f'GTF feather 파일이 없습니다: {GTF_FEATHER_PATH}'
    )

df = pd.read_feather(GTF_FEATHER_PATH)
print(df.columns)


Index(['Chromosome', 'Source', 'Feature', 'Start', 'End', 'Score', 'Strand',
       'Frame', 'gene_id', 'gene_type', 'gene_name', 'level', 'tag',
       'transcript_id', 'transcript_type', 'transcript_name',
       'transcript_support_level', 'havana_transcript', 'exon_number',
       'exon_id', 'hgnc_id', 'havana_gene', 'ont', 'protein_id', 'ccdsid',
       'artif_dupl', 'gene_id_nopatch'],
      dtype='object')


In [11]:
chrom_col = 'Chromosome' if 'Chromosome' in df.columns else 'chromosome'
start_col = 'Start' if 'Start' in df.columns else 'start'
end_col = 'End' if 'End' in df.columns else 'end'
strand_col = 'Strand' if 'Strand' in df.columns else 'strand'

hbb_gene = df[(df['gene_name'] == GENE_NAME) & (df['Feature'] == 'gene')].iloc[0]

gene_id = hbb_gene['gene_id']
chrom = hbb_gene[chrom_col]
# # retrieve from ensembl regulator build.
# start = int(hbb_gene[start_col])
# end = int(hbb_gene[end_col])
start=5225464
end=5227071
strand = hbb_gene[strand_col]

print(f"{GENE_NAME} Gene({gene_id}): {chrom}:{start}-{end} ({strand})")


HBB Gene(ENSG00000244734.4): chr11:5225464-5227071 (-)


In [12]:
if strand == '+':
    tss = start
    promoter_start = tss - PROMOTER_TSS_LENGTH
    promoter_end = tss # 이러면 TSS 시작하는 1bp가 겹침
    # promoter_end = tss - 1 # 안 겹침. upstream-only
else:
    tss = end
    promoter_start = tss
    promoter_end = tss + PROMOTER_TSS_LENGTH

print(f"Calculated Promoter Region (hg38): {chrom}:{promoter_start}-{promoter_end}")


Calculated Promoter Region (hg38): chr11:5227071-5228071


In [13]:
def get_sequence_from_ensembl_region(species, chrom, start, end, strand, coord_system_version=None):
    """Half-open interval [start, end) in 1-based coordinates.

    Ensembl REST expects inclusive end, so we convert end -> end-1.
    """
    if end <= start:
        raise ValueError(f'Invalid half-open interval: start={start}, end={end}')

    server = 'https://rest.ensembl.org'
    clean_chrom = str(chrom).replace('chr', '')
    strand_val = 1 if str(strand) in ['+', '1'] else -1
    inclusive_end = end - 1
    region = f"{clean_chrom}:{start}..{inclusive_end}:{strand_val}"
    ext = f"/sequence/region/{species}/{region}"
    if coord_system_version:
        ext = f"{ext}?coord_system_version={coord_system_version}"

    headers = {'Content-Type': 'text/plain'}
    r = requests.get(server + ext, headers=headers)
    r.raise_for_status()
    return r.text.strip()


def normalize_sequence(seq):
    seq = seq.upper()
    normalized = []
    for ch in seq:
        if ch in 'ACGT':
            normalized.append(ch)
        else:
            normalized.append('N')
    return ''.join(normalized)


promoter_seq_raw = get_sequence_from_ensembl_region(
    ENSEMBL_SPECIES,
    chrom,
    promoter_start,
    promoter_end,
    strand,
    coord_system_version=ENSEMBL_ASSEMBLY,
)

promoter_seq = normalize_sequence(promoter_seq_raw)
print(f'Promoter sequence length: {len(promoter_seq)}')
print(promoter_seq[:60] + '...')


Promoter sequence length: 1000
GAACTTGAATCAAGGAAATGATTTTAAAACGCAGTATTCTTAGTGGACTAGAGGAAAAAA...


In [ ]:
gene_selection = {f'{GENE_NAME}_promoter': promoter_seq}
print(gene_selection.keys)


<built-in method keys of dict object at 0x15cc1d500>


: 

In [15]:
device = get_device()
print(f'사용 중인 디바이스: {device}')

print('시퀀스 생성 중...')
for model_label, model_instance in iter_models(
    device, model_configs, torch_dtype=torch_dtype
):
    print(f'모델: {model_label}')

    model_name = model_label.replace('/', '-')
    model_dir = PROMOTER_DIR / model_name
    model_dir.mkdir(parents=True, exist_ok=True)

    output_csv = model_dir / f'{GENE_NAME}_promoter.csv'
    results_data = {}

    for strategy_base_key, strategy_cfg in DEFAULT_DECODING_STRATEGIES.items():
        strategy_type = strategy_cfg['type']
        temperatures = strategy_cfg.get('temperatures', [1.0])
        top_k = strategy_cfg.get('top_k', 50)

        for temp in temperatures:
            if strategy_type == 'greedy':
                strategy_key = strategy_base_key
            else:
                strategy_key = f'{strategy_base_key}_t{temp}'

            generated_sequences = model_instance.run(
                sequence=promoter_seq,
                steps=ITERATIONS,
                mask_ratio=MASK_RATIO,
                strategy=strategy_type,
                temperature=temp,
                top_k=top_k,
                save_all=True,
                save_interval=1,
            )

            results_data[strategy_key] = generated_sequences
            del generated_sequences
            gc.collect()
            if model_instance.device == 'cuda':
                torch.cuda.empty_cache()
            elif model_instance.device == 'mps':
                torch.mps.empty_cache()

    df = pd.DataFrame(results_data).T
    df.columns = [f'iteration_{i}' for i in range(df.shape[1])]
    df.to_csv(output_csv)
    print(f'✅ 저장 완료: {output_csv}')


PyTorch Version: 2.9.1
Using device: mps
사용 중인 디바이스: mps
시퀀스 생성 중...
📥 Downloading DNABERT-2...


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

✅ DNABERT-2 Triton patch applied successfully.
[DNABERT-2] Loading model...


/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:126: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(


[DNABERT-2] Model loaded successfully.
✅ DNABERT-2 loaded successfully.
모델: DNABERT-2
✅ 저장 완료: ../data/promoters/DNABERT-2/HBB_promoter.csv
[NT-v2-50m] Loading model...
[NT-v2-50m] Model loaded successfully.
✅ NT-v2-50m loaded successfully.
모델: NT-v2-50m
✅ 저장 완료: ../data/promoters/NT-v2-50m/HBB_promoter.csv
[NT-v2-500m] Loading model...
[NT-v2-500m] Model loaded successfully.
✅ NT-v2-500m loaded successfully.
모델: NT-v2-500m
✅ 저장 완료: ../data/promoters/NT-v2-500m/HBB_promoter.csv
